In [18]:
import pandas
import pandas as pd
import numpy as np
import sys
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names
# from utilis import get_feat_dict

In [1]:
def get_feat_dict(data):
    if data == 'ml-1m':
        feat_dict = {'movie_id': 3706,
                 'user_id': 6040,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'zipcode': 3439}
    if data == 'black-friday':
        feat_dict = {'user_id': 5891,
                 'product_id': 3631,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'city': 3,
                 'stay_years': 5,
                 'martial_status': 2,
                 'cate_1': 20,
                 'cate_2': 18,
                 'cate_3': 16,
                 'price': 23961 }
    return feat_dict

In [14]:
#load data

saved_model_path = '/data/linghui/saved_model/deepfm-ml-1m/'

saved_data_path = '/data/linghui/ml-1m/processed_data/'
# train_data_path = saved_data_path + 'train_data.csv' 
# test_data_path = saved_data_path + 'test_data.csv'
train_data_path = saved_data_path + 'train_data_liuyi.csv' 
test_data_path = saved_data_path + 'test_data_liuyi.csv'


#load data
train_data = pd.read_csv(train_data_path,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
train_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,0,2969,1,978300019,"Face in the Crowd, A (1957)",Drama,0,0,10,1588
1,0,1178,1,978300055,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,0,0,10,1588
2,0,1574,1,978300055,"Peacemaker, The (1997)",Action|Thriller|War,0,0,10,1588
3,0,957,1,978300055,"African Queen, The (1951)",Action|Adventure|Romance|War,0,0,10,1588
4,0,2147,1,978300103,"Skin Game, The (1931)",Drama,0,0,10,1588
...,...,...,...,...,...,...,...,...,...,...
1988333,6039,1074,0,0,Platoon (1986),Drama|War,1,2,6,466
1988334,6039,627,0,0,Land and Freedom (Tierra y libertad) (1995),War,1,2,6,466
1988335,6039,2882,0,0,"Fistful of Dollars, A (1964)",Action|Western,1,2,6,466
1988336,6039,3356,0,0,Mo' Better Blues (1990),Drama,1,2,6,466


In [ ]:
from ast import literal_eval

genres_list = train_data['genres'].tolist()
genres = []
for i in range(len(genres_list)):
    genres.append(literal_eval(genres_list[i]))
train_data['genres'] = genres


In [7]:
dataset = 'ml-1m'
feat_dict = get_feat_dict(dataset)
feat_dict

{'movie_id': 3706,
 'user_id': 6040,
 'gender': 2,
 'age': 7,
 'occupation': 21,
 'zipcode': 3439}

In [48]:
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode", ]

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in sparse_features]

In [16]:
movie_size = 18
max_len = 6

train_feats = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode"]

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in train_feats]

# use_weighted_sequence = False
# if use_weighted_sequence:
#     varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size = movie_size + 1, 
#                                                           embedding_dim=4), maxlen=max_len, combiner='mean', weight_name='genres_weight')]  
#     # Notice : value 0 is for padding for sequence input feature
# else:
#     varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size = movie_size + 1, 
#                                                           embedding_dim=4), maxlen=max_len, combiner='mean', weight_name=None)]  
    # Notice : value 0 is for padding for sequence input feature

# linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
# dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [19]:
train_data = sklearn.utils.shuffle(train_data)

train_model_input = {name: train_data[name].values for name in train_feats}
label = train_data['rating'].values

In [20]:
train_model_input

{'movie_id': array([1807, 2091, 1201, ...,  248, 1185, 1509]),
 'user_id': array([2933, 5039, 2941, ..., 5328, 5830, 3128]),
 'gender': array([0, 1, 0, ..., 0, 1, 1]),
 'age': array([3, 4, 5, ..., 2, 2, 2]),
 'occupation': array([20, 12,  1, ...,  5,  1,  7]),
 'zipcode': array([2839, 1451, 3003, ..., 2876, 2924,  845])}

In [21]:
label

array([0, 0, 1, ..., 0, 0, 0])

In [22]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')

2022-05-30 10:56:40.738916: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-30 10:56:40.754593: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-05-30 10:56:40.756143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-05-30 10:56:40.756179: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-30 10:56:40.758167: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library l

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
              metrics=['AUC', 'Precision', 'Recall'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

In [24]:
batch_size = 256
epochs = 20

check_path = saved_model_path + 'deepfm-ml-1m-liuyi.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss',
                                                 )

In [25]:
print('begain training')
model.fit(train_model_input, label,
						batch_size=batch_size, epochs=epochs, verbose=2,
                        callbacks=[checkpoint],
						validation_split=0.2)

begain training
Epoch 1/20


/opt/conda/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-05-30 10:58:06.814982: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


6214/6214 - 43s - loss: 0.4822 - auc: 0.8470 - precision: 0.7462 - recall: 0.8005 - val_loss: 0.4677 - val_auc: 0.8585 - val_precision: 0.7710 - val_recall: 0.7837
Epoch 2/20
6214/6214 - 43s - loss: 0.4481 - auc: 0.8720 - precision: 0.7696 - recall: 0.8261 - val_loss: 0.4429 - val_auc: 0.8761 - val_precision: 0.7728 - val_recall: 0.8313
Epoch 3/20
6214/6214 - 43s - loss: 0.4269 - auc: 0.8866 - precision: 0.7848 - recall: 0.8388 - val_loss: 0.4201 - val_auc: 0.8919 - val_precision: 0.7983 - val_recall: 0.8307
Epoch 4/20
6214/6214 - 43s - loss: 0.4077 - auc: 0.8990 - precision: 0.7992 - recall: 0.8485 - val_loss: 0.4098 - val_auc: 0.8990 - val_precision: 0.8048 - val_recall: 0.8419
Epoch 5/20
6214/6214 - 43s - loss: 0.3993 - auc: 0.9047 - precision: 0.8057 - recall: 0.8539 - val_loss: 0.4070 - val_auc: 0.9017 - val_precision: 0.8034 - val_recall: 0.8537
Epoch 6/20
6214/6214 - 43s - loss: 0.3956 - auc: 0.9075 - precision: 0.8091 - recall: 0.8560 - val_loss: 0.4046 - val_auc: 0.9033 - val_